In [2]:
%matplotlib inline
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim

In [3]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [16]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)
shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices

(tensor([ 8,  4, 10,  6,  9,  7,  1,  3,  5]), tensor([2, 0]))

In [17]:
train_t_u = t_u[train_indices]
train_t_c = t_c[train_indices]

val_t_u = t_u[val_indices]
val_t_c = t_c[val_indices]

train_t_un = train_t_u * 0.1
val_t_un = val_t_u  * 0.1

train_indices, val_indices

(tensor([ 8,  4, 10,  6,  9,  7,  1,  3,  5]), tensor([2, 0]))

In [18]:
def model(t_u, w, b):
    return w * t_u + b

def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c) ** 2
    return squared_diffs.mean()

In [24]:
def training_loop(n_epochs, optimizer, params, train_t_u, val_t_u, train_t_c, val_t_c):
    for epoch in range(1, n_epochs + 1):
        if params.grad is not None:
            params.grad.zero_()
        train_t_p = model(train_t_u, *params)
        train_loss = loss_fn(train_t_p, train_t_c)

        val_t_p = model(val_t_u, *params)
        val_loss = loss_fn(val_t_p, val_t_c)

        # don't zero validation grad
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        with torch.no_grad():
            params -= learning_rate * params.grad

        if epoch%500 == 0:
            print('Epoch {}, loss: {}'.format(epoch, float(train_loss)))
    return params

In [25]:
params = torch.tensor([1.0 , 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)

res = training_loop(
    n_epochs=3000,
    optimizer=optimizer,
    params=torch.tensor([1.0, 0.0], requires_grad=True),
    train_t_u = train_t_un,
    val_t_u = val_t_un,
    train_t_c = train_t_c,
    val_t_c = val_t_c,
)

Epoch 500, loss: 7.6495795249938965
Epoch 1000, loss: 3.964651346206665
Epoch 1500, loss: 3.3377959728240967
Epoch 2000, loss: 3.231158971786499
Epoch 2500, loss: 3.2130212783813477
Epoch 3000, loss: 3.2099361419677734


In [26]:
res

tensor([  5.2384, -16.5813], requires_grad=True)